**동물 분류 문제**

**동물의 특징에 따른 7 가지 분류**

[1] 데이터 로드

In [0]:
!curl https://raw.githubusercontent.com/sejongresearch/2020.Spring.AI/master/HW/zoo_class.csv -o zoo_class.csv
print('Done..\n')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3535  100  3535    0     0  20085      0 --:--:-- --:--:-- --:--:-- 20085
Done..



In [0]:
import torch
import torch.optim as optim 
import torch.nn.functional as F 
# 소프트맥스 함수를 사용하기 위함. 
import numpy as np 
# numpy 
import pandas as pd
# pandas

torch.manual_seed(1)

data_zoo = pd.read_csv('zoo_class.csv')
data_zoo.head()
#print(len(data_zoo))

x_data = data_zoo.iloc[:,:-1]
#x_data.head()

y_data = data_zoo.iloc[:,[-1]]
#y_data.head()

print(x_data.shape)
print(y_data.shape)

x_train = torch.FloatTensor(x_data.values)
y_train = torch.LongTensor(y_data.values)

print(x_train.shape)
y_train = y_train.view(100)
print(y_train.shape)

(100, 16)
(100, 1)
torch.Size([100, 16])
torch.Size([100])


[2] 모델 학습 

In [0]:
nb_class = 7
nb_data = len(y_train)

w = torch.zeros((16, nb_class),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b], lr = 0.05)


nb_epochs = 200000

for epoch in range(nb_epochs+1):

    hypothesis = F.softmax(x_train.matmul(w)+b, dim=1) # 소프트맥스 함수 / 가설함수 

    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10000 == 0:
        print('epoch {:4d}/{} cost : {:.6f}'.format(
            epoch, nb_epochs,cost.item()
        ))

epoch    0/100000 cost : 1.945910
epoch 10000/100000 cost : 1.317136
epoch 20000/100000 cost : 1.296551
epoch 30000/100000 cost : 1.288721
epoch 40000/100000 cost : 1.284782
epoch 50000/100000 cost : 1.214139
epoch 60000/100000 cost : 1.210443
epoch 70000/100000 cost : 1.208711
epoch 80000/100000 cost : 1.207669
epoch 90000/100000 cost : 1.206963
epoch 100000/100000 cost : 1.206439


[3] 데이터 정확도

In [0]:
hypothesis = F.softmax(x_train.matmul(w) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)
#print(hypothesis)
#print(predict)
#print(y_train)

# 정확도 계산 
correct_prediction = predict.float() == y_train
#print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


The model has an accuracy of 96.00% for the training set.
